In [1]:
import pandas as pd

In [2]:
s = pd.read_parquet('rassp-public/sample_data/smallmols_cfm_pred_public_sample.parquet')

In [7]:
s.iloc[0].spect

array([array([15.02292522,  0.91388646]),
       array([25.00727602,  0.96256718]),
       array([27.02292522,  3.49654964]),
       array([29.03857442,  2.88757419]),
       array([39.02292522,  0.92641582]),
       array([41.03857442,  3.63968496]),
       array([42.04639902,  2.29737418]),
       array([43.05422362,  8.79653005]),
       array([43.97152142,  2.10668418]),
       array([44.06204822,  1.32544061]),
       array([44.97934602,  2.70170413]),
       array([45.06987282,  1.27590546]),
       array([55.05422362,  2.72504282]),
       array([58.98241922,  4.22013733]),
       array([59.99024382,  2.07733632]),
       array([69.06987282,  2.20343483]),
       array([70.07769742,  1.42581885]),
       array([71.08552202,  9.44191393]),
       array([84.99806842,  1.50804528]),
       array([86.00589302, 18.14332272]),
       array([87.01371762,  1.56864662]),
       array([88.02154222,  1.06527212]),
       array([114.0371914 ,   2.45328396]),
       array([129.0606652 ,   2.

In [8]:
s.iloc[0].spect_sparse

array([array([15.        ,  0.91388646]),
       array([25.        ,  0.96256718]),
       array([27.        ,  3.49654964]),
       array([29.        ,  2.88757419]),
       array([39.        ,  0.92641582]),
       array([41.        ,  3.63968496]),
       array([42.        ,  2.29737418]),
       array([43.        ,  8.79653005]),
       array([44.        ,  2.10668418]),
       array([44.        ,  1.32544061]),
       array([45.        ,  2.70170413]),
       array([45.        ,  1.27590546]),
       array([55.        ,  2.72504282]),
       array([59.        ,  4.22013733]),
       array([60.        ,  2.07733632]),
       array([69.        ,  2.20343483]),
       array([70.        ,  1.42581885]),
       array([71.        ,  9.44191393]),
       array([85.        ,  1.50804528]),
       array([86.        , 18.14332272]),
       array([87.        ,  1.56864662]),
       array([88.        ,  1.06527212]),
       array([114.        ,   2.45328396]),
       array([129.        ,   2.

In [9]:
s.iloc[0]

mol_id                                              NIST2014_10006
inchi                InChI=1S/C6H11NS/c1-5(2)6(3)7-4-8/h5-6H,1-3H3
smiles                                              CC(C)C(C)N=C=S
rdmol            b'\xef\xbe\xad\xde\x00\x00\x00\x00\x0c\x00\x00...
cv_id                                                            7
morgan4_crc32                                           3981836209
spect            [[15.02292522, 0.9138864566], [25.00727602, 0....
spect_sparse     [[15.0, 0.9138864566], [25.0, 0.9625671785], [...
inchi_key                              YAVRRVJBEIYPRH-UHFFFAOYSA-N
Name: 0, dtype: object